In [7]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine
import json

In [2]:
usuarios_columnas = ["id_usuario", "genero_usuario", "edad_usuario", "fecha_alta"]
carpeta = "usuarios"
dataframes = []

for archivo in os.listdir(carpeta):
    if archivo.endswith(".csv") and "usuarios" in archivo:
        ruta_archivo = os.path.join(carpeta, archivo)
        df = pd.read_csv(ruta_archivo)
        # Estandarizar nombres de columnas
        df.columns = [col.lower().replace('"', '') for col in df.columns]
        # Convertir a minuscula los headers para estandarizar
        df = df.rename(columns={col.lower(): col for col in usuarios_columnas})
        # Selecciona solo las columnas deseadas
        df = df[usuarios_columnas]
        # Estandarizar el formato de la fecha
        df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], errors="coerce").dt.strftime("%Y-%m-%d")
        dataframes.append(df)

usuarios_df = pd.concat(dataframes, ignore_index=True)

# Renombrar columnas
usuarios_df = usuarios_df.rename(columns={"genero_usuario":"genero", "edad_usuario":"edad"})
# Convertir nombres de columnas a minúsculas
usuarios_df.columns = map(str.lower, usuarios_df.columns)
# Eliminar los espacios que puedan tener los strings al principio o al final
usuarios_df = usuarios_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Guardar solo la inicial de genero para mayor eficiencia. Los valores serán M = MALE, F = FEMALE, O = OTHER
usuarios_df["genero"] = usuarios_df["genero"].str[0]
display(usuarios_df)
# Asegurarse de que la columna generosolo contiene los valores válidos
valores_genero = usuarios_df["genero"].unique()
print(f"Valores genero: {valores_genero}")
# Revisar si hay valores nulos
valores_nulos = usuarios_df.isnull().values.any()
print(f"Valores nulos: {valores_nulos}")
# Eliminar las filas donde edad contenga "," ya que solo se permiten numeros enteros
usuarios_df = usuarios_df[~usuarios_df["edad"].astype(str).str.contains(",")]
# Eliminar las filas con edad NaN
usuarios_df = usuarios_df.dropna(subset=["edad"])
# Convertir a int
usuarios_df["edad"] = usuarios_df["edad"].astype(int)
# Chequear inconsistencias en los valores de edad
edad_inconsistente = usuarios_df[(usuarios_df["edad"] < 10) | (usuarios_df["edad"] > 125)]
display(edad_inconsistente)
# Eliminar las filas con edad incosistente
usuarios_df = usuarios_df[(usuarios_df["edad"] >= 10) & (usuarios_df["edad"] <= 125)]
display(usuarios_df)

print(f"id_usuarios: {usuarios_df['id_usuario'].dtype}")
print(f"genero: {usuarios_df['genero'].dtype}")
print(f"edad: {usuarios_df['edad'].dtype}")
print(f"fecha_alta: {usuarios_df['fecha_alta'].dtype}")

# Ordenar for fecha para eliminar los registros con id_usuario duplicado y mantener los que tengan mayor fecha
usuarios_df = usuarios_df.sort_values(by="fecha_alta", ascending=False)
# Mostrar duplicados
display(usuarios_df[usuarios_df["id_usuario"].duplicated(keep=False)])
usuarios_df = usuarios_df.drop_duplicates(subset="id_usuario", keep="first")
display(usuarios_df)


C:\Users\lauta\AppData\Local\Temp\ipykernel_20840\302955926.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], errors="coerce").dt.strftime("%Y-%m-%d")
C:\Users\lauta\AppData\Local\Temp\ipykernel_20840\302955926.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], errors="coerce").dt.strftime("%Y-%m-%d")
C:\Users\lauta\AppData\Local\Temp\ipykernel_20840\302955926.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], err

,id_usuario,genero,edad,fecha_alta
0,7682,M,45,2015-02-28
1,19577,M,47,2015-02-28
2,108635,M,26,2015-02-28
3,129948,M,29,2015-02-28
4,129949,M,24,2015-02-28
...,...,...,...,...
708683,940015,F,51,2023-01-01
708684,939856,O,29,2023-01-01
708685,940052,M,53,2023-01-01
708686,940041,F,37,2023-01-01


Valores genero: ['M' 'F' nan 'O']
Valores nulos: True


,id_usuario,genero,edad,fecha_alta
140135,601777,F,160,2018-08-30
143455,344432,M,0,2018-09-13
143582,220584,F,126,2018-12-19
144703,602312,M,129,2018-08-29
152787,404308,M,140,2018-03-09
...,...,...,...,...
674128,979498,O,955,2023-04-22
697001,953240,O,967,2023-02-04
703840,945547,O,951,2023-01-14
706247,942804,O,155,2023-01-08


,id_usuario,genero,edad,fecha_alta
0,7682,M,45,2015-02-28
1,19577,M,47,2015-02-28
2,108635,M,26,2015-02-28
3,129948,M,29,2015-02-28
4,129949,M,24,2015-02-28
...,...,...,...,...
708683,940015,F,51,2023-01-01
708684,939856,O,29,2023-01-01
708685,940052,M,53,2023-01-01
708686,940041,F,37,2023-01-01


id_usuarios: int64
genero: object
edad: int32
fecha_alta: object


,id_usuario,genero,edad,fecha_alta
340844,672918,F,24,2020-07-24
340850,672938,F,51,2020-07-24
340774,672973,M,23,2020-07-24
340775,672933,O,35,2020-07-24
340760,672922,O,32,2020-07-24
...,...,...,...,...
10901,156917,F,57,2015-01-05
10902,160084,M,48,2015-01-05
10908,159605,F,29,2015-01-05
10912,148093,M,27,2015-01-05


,id_usuario,genero,edad,fecha_alta
572795,1083497,M,20,2023-12-31
572938,1083420,F,18,2023-12-31
572936,1083117,F,23,2023-12-31
572935,1083289,M,24,2023-12-31
572934,1083342,O,24,2023-12-31
...,...,...,...,...
10907,145180,F,31,2015-01-05
10909,159761,M,26,2015-01-05
10911,146257,M,30,2015-01-05
10913,160058,F,20,2015-01-05


In [9]:
estaciones_columnas = ["ID Comet", "NÚMERO de Estación ", "NOMBRE", "BARRIO", "COMUNA"]
estaciones_df = pd.read_csv("nuevas-estaciones-bicicletas-publicas.csv", encoding="latin-1", delimiter=";", usecols=estaciones_columnas)
# Eliminar los espacios que puedan tener los strings al principio o al final, tanto en filas como en columnas
estaciones_df.columns = estaciones_df.columns.str.strip()
estaciones_df = estaciones_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Renombrar columnas
estaciones_df = estaciones_df.rename(columns={"ID Comet":"id_estacion", "NÚMERO de Estación":"numero_estacion"})

# Convertir nombres de columnas a minúsculas
estaciones_df.columns = map(str.lower, estaciones_df.columns)
# Quedarse solo con el número de la columna comuna
estaciones_df["comuna"] = estaciones_df["comuna"].str.extract(r'(\d+)')
display(estaciones_df)
# Revisar si hay valores nulos
valores_nulos = estaciones_df.isnull().values.any()
print(f"Valores nulos: {valores_nulos}")
# Convertir a int
estaciones_df["comuna"] = estaciones_df["comuna"].astype(int)

print(f"id_estacion: {estaciones_df['id_estacion'].dtype}")
print(f"numero_estacion: {estaciones_df['numero_estacion'].dtype}")
print(f"nombre: {estaciones_df['nombre'].dtype}")
print(f"barrio: {estaciones_df['barrio'].dtype}")
print(f"comuna: {estaciones_df['comuna'].dtype}")

     id_estacion  numero_estacion         NOMBRE        BARRIO     COMUNA
295          471              173  EL LIBERTADOR  VILLA DEVOTO  COMUNA 11


,id_estacion,numero_estacion,nombre,barrio,comuna
0,2,2,RETIRO I,RETIRO,1
1,3,3,ADUANA,MONSERRAT,1
2,4,4,PLAZA ROMA,SAN NICOLAS,1
3,5,5,PLAZA ITALIA,PALERMO,14
4,6,6,PARQUE LEZAMA,SAN TELMO,1
...,...,...,...,...,...
355,532,72,INSTITUTO POLITECNICO MODELO,VILLA PUEYRREDON,12
356,533,77,CRISOLOGO LARRALDE,SAAVEDRA,12
357,534,81,PARQUE DE LAS CIENCIAS,PALERMO,14
358,535,90,HONORIO PUEYRREDON,VILLA CRESPO,15


Valores nulos: False
id_estacion: int64
numero_estacion: int64
nombre: object
barrio: object
comuna: int32


In [4]:
viajes_columnas = ["Id_recorrido","duracion_recorrido","id_estacion_origen","id_estacion_destino","id_usuario","modelo_bicicleta"]
viajes_df = pd.read_csv("trips_2023.csv", usecols=viajes_columnas)
# Renombrar la columna duracion_recorrido por duracion_recorrido(seg)
viajes_df = viajes_df.rename(columns={"duracion_recorrido": "duracion_segundos"})
# Convertir nombres de columnas a minúsculas
viajes_df.columns = map(str.lower, viajes_df.columns)
# Eliminar "BAEcobici" de las columnas Id_recorrido, id_estacion_origen, id_estacion_destino y id_usuario para poder hacer las conexiones
viajes_df["id_recorrido"] = viajes_df["id_recorrido"].str.replace("BAEcobici", "")
viajes_df["id_estacion_origen"] = viajes_df["id_estacion_origen"].str.replace("BAEcobici", "")
viajes_df["id_estacion_destino"] = viajes_df["id_estacion_destino"].str.replace("BAEcobici", "")
viajes_df["id_usuario"] = viajes_df["id_usuario"].str.replace("BAEcobici", "")
display(viajes_df)
# Ver que valores puede traer la columna modelo_bicicleta para saber con que longitud almacenarla en la BD
valores_modelo = viajes_df["modelo_bicicleta"].unique()
print(f"Valores modelo bicicleta: {valores_modelo}")
# Revisar si hay valores nulos
valores_nulos = viajes_df[viajes_df.isnull().values.any(axis=1)]
display(valores_nulos) #  hay 2 filas sin información sobre el destino del viaje
# Eliminar las filas con valores nulos
viajes_df = viajes_df.dropna()
# Quitar las comas de la columna duracion_segundos
viajes_df["duracion_segundos"] = viajes_df["duracion_segundos"].str.replace(",", "")

display(viajes_df)

# Convertir tipos de datos de las columnas para que coincidan con los que requieren la tabla de postgreSQL
viajes_df["id_recorrido"] = viajes_df["id_recorrido"].astype(int)
viajes_df["duracion_segundos"] = viajes_df["duracion_segundos"].astype(int)
viajes_df["id_estacion_origen"] = viajes_df["id_estacion_origen"].astype(int)
viajes_df["id_estacion_destino"] = viajes_df["id_estacion_destino"].astype(int)
viajes_df["id_usuario"] = viajes_df["id_usuario"].astype(int)

print(f"id_recorrido: {viajes_df['id_recorrido'].dtype}")
print(f"duracion_segundos: {viajes_df['duracion_segundos'].dtype}")
print(f"id_estacion_origen: {viajes_df['id_estacion_origen'].dtype}")
print(f"id_estacion_destino: {viajes_df['id_estacion_destino'].dtype}")
print(f"id_usuario: {viajes_df['id_usuario'].dtype}")
print(f"modelo_bicicleta: {viajes_df['modelo_bicicleta'].dtype}")

,id_recorrido,duracion_segundos,id_estacion_origen,id_estacion_destino,id_usuario,modelo_bicicleta
0,17910696,"1,848",358,278,861866,ICONIC
1,17600256,288,444,3,217525,ICONIC
2,17255670,"1,103",280,280,954201,ICONIC
3,17996972,"1,165",273,367,179414,ICONIC
4,17148836,378,65,14,8098,ICONIC
...,...,...,...,...,...,...
2622326,19981607,415,14,516,330925,FIT
2622327,19982589,"1,714",516,516,56874,FIT
2622328,19982034,"2,191",187,518,1074657,FIT
2622329,19978634,"4,112",524,524,766418,FIT


Valores modelo bicicleta: ['ICONIC' 'FIT']


,id_recorrido,duracion_segundos,id_estacion_origen,id_estacion_destino,id_usuario,modelo_bicicleta
2111365,19632972,"1,372",26,NaN,1020610,FIT
2177905,19632583,466,38,NaN,774845,FIT


,id_recorrido,duracion_segundos,id_estacion_origen,id_estacion_destino,id_usuario,modelo_bicicleta
0,17910696,1848,358,278,861866,ICONIC
1,17600256,288,444,3,217525,ICONIC
2,17255670,1103,280,280,954201,ICONIC
3,17996972,1165,273,367,179414,ICONIC
4,17148836,378,65,14,8098,ICONIC
...,...,...,...,...,...,...
2622326,19981607,415,14,516,330925,FIT
2622327,19982589,1714,516,516,56874,FIT
2622328,19982034,2191,187,518,1074657,FIT
2622329,19978634,4112,524,524,766418,FIT


id_recorrido: int32
duracion_segundos: int32
id_estacion_origen: int32
id_estacion_destino: int32
id_usuario: int32
modelo_bicicleta: object


In [5]:
# Asegurarse que las FK de viajes existan en las tablas usuarios y estaciones
# Obtener la lista de valores de id_usuario del df de usuarios
usuarios_existentes = usuarios_df["id_usuario"].tolist()
estaciones_existentes = estaciones_df["id_estacion"].tolist()
# Filtrar el df de viajes
viajes_filtrado = viajes_df[
    (viajes_df["id_usuario"].isin(usuarios_existentes)) &
    (viajes_df["id_estacion_origen"].isin(estaciones_existentes)) &
    (viajes_df["id_estacion_destino"].isin(estaciones_existentes))
]
display(viajes_filtrado)


,id_recorrido,duracion_segundos,id_estacion_origen,id_estacion_destino,id_usuario,modelo_bicicleta
0,17910696,1848,358,278,861866,ICONIC
1,17600256,288,444,3,217525,ICONIC
2,17255670,1103,280,280,954201,ICONIC
3,17996972,1165,273,367,179414,ICONIC
4,17148836,378,65,14,8098,ICONIC
...,...,...,...,...,...,...
2622324,19982650,1218,469,469,1074664,FIT
2622325,19980512,1300,267,473,1074463,FIT
2622327,19982589,1714,516,516,56874,FIT
2622328,19982034,2191,187,518,1074657,FIT


In [6]:
# Leer el archivo de configuración
with open("config.json") as f:
    config = json.load(f)

# Acceder a las configuraciones
user = config["DB_USER"]
password = config["DB_PASSWORD"]
host = config["DB_HOST"]
database = config["DB_DATABASE"]
port = config["DB_PORT"]

# Esquema de las tablas
usuarios_schema = "id_usuario INT PRIMARY KEY, genero CHAR(1), edad SMALLINT, fecha_alta DATE"
estaciones_schema = "id_estacion INT PRIMARY KEY, numero_estacion INT, nombre VARCHAR(70), barrio VARCHAR(50), comuna SMALLINT"
viajes_schema = "id_recorrido INT PRIMARY KEY, duracion_segundos INT, id_estacion_origen INT REFERENCES estaciones(id_estacion), id_estacion_destino INT REFERENCES estaciones(id_estacion), id_usuario INT REFERENCES usuarios(id_usuario), modelo_bicicleta VARCHAR(10)"

# Conexión a la base de datos
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Eliminar los registros existentes en las tablas
tablas = ["viajes", "usuarios", "estaciones"]
with conn.cursor() as cursor:
    for tabla in tablas:
        cursor.execute(f"DROP TABLE IF EXISTS {tabla}")

    cursor.execute(f"CREATE TABLE usuarios ({usuarios_schema})")
    cursor.execute(f"CREATE TABLE estaciones ({estaciones_schema})")
    cursor.execute(f"CREATE TABLE viajes ({viajes_schema})")
    cursor.execute("CREATE INDEX idx_id_usuario ON viajes (id_usuario)")
    cursor.execute("CREATE INDEX idx_id_estacion_origen ON viajes (id_estacion_origen)")
    cursor.execute("CREATE INDEX idx_id_estacion_destino ON viajes (id_estacion_destino)")

conn.commit()  # Confirmar y aplicar la transacción

usuarios_df.to_sql("usuarios", engine, if_exists="append", index=False)
estaciones_df.to_sql("estaciones", engine, if_exists="append", index=False)
viajes_filtrado.to_sql("viajes", engine, if_exists="append", index=False)

conn.close()